In [52]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import requests
from bs4 import BeautifulSoup
import re
import numpy as np
import pandas as pd

In [4]:
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

# Test for one sentence

In [5]:
tokens = tokenizer.encode('its ok', return_tensors='pt')


In [6]:
result = model(tokens)

In [7]:
torch.argmax(result.logits)

tensor(2)

In [8]:
int(torch.argmax(result.logits))+1

3

# Collect reviews

In [49]:
url = 'https://www.lookfantastic.com.hk/aesop-parsley-seed-anti-oxidant-serum-100ml/10363854.html'
r = requests.get('http://localhost:8050/render.html', params = {'url':url,'wait':2})
soup = BeautifulSoup(r.text, 'html.parser')
results = soup.find_all('p',{'class':'productReviews_topReviewsExcerpt'})
reviews = [result.text for result in results]

In [50]:
reviews

['\nJust one use, I can feel the difference the next day. Brighter with moisture. This protection layer is required I recommend.\n',
 '\nGreat product, on my third bottle! My skin feels more resistant and does not get sensitive or break out as easily. \n',
 '\nFirst time using Aesop & this serum. Not oily easily absorb , use together with Aesop Lucent day & night.\xa0\nFace seems clearer after a 2 weeks. Not sure it’s due to this Parsley serum or the Lucent serum.\xa0\nHappy with purchase especially when it’s on discount t\n\n',
 '\nThis is my holy grail serum\nAbsorbs fast and keeps skin plump and soft like a baby’s skin\n',
 '\nUse after few weeks, I feel my skin brightening than before. It really good scent and moisturiser.\nYou can wait it discount and buy to try. Look fantastic really more often give discount on this brand. Delivery fast to me. (Malaysia)\n',
 "\nI've been using this product for a few years now, bought this on lookfantastic when there was an offer so the price was

# Score

In [53]:
df = pd.DataFrame(np.array(reviews), columns=['review'])


In [54]:
df['review'].iloc[0]


'\nJust one use, I can feel the difference the next day. Brighter with moisture. This protection layer is required I recommend.\n'

In [55]:
def sentiment_score(review):
    tokens = tokenizer.encode(review, return_tensors='pt')
    result = model(tokens)
    return int(torch.argmax(result.logits))+1

In [56]:
sentiment_score(df['review'].iloc[1])


5

In [57]:
df['sentiment'] = df['review'].apply(lambda x: sentiment_score(x[:512]))


In [58]:
df

,review,sentiment
0,"\nJust one use, I can feel the difference the ...",5
1,"\nGreat product, on my third bottle! My skin f...",5
2,\nFirst time using Aesop & this serum. Not oil...,5
3,\nThis is my holy grail serum\nAbsorbs fast an...,5
4,"\nUse after few weeks, I feel my skin brighten...",5
5,\nI've been using this product for a few years...,4
6,\nthis serum suits my skin condition. (Combina...,5
7,\nI have combination skin and product is certa...,4
